**获取配点**

In [86]:
%reset -f

In [87]:
import sys, os
sys.path.append(os.path.dirname(os.path.realpath('../utils')))
import numpy as np
from utils.iwavelets import xphi0

In [88]:
j0 = 0
J = 15
a = 0
b = 2

eps0 = 0.005

deltax = (b-a)/(2**J)
XJ0 = np.arange(a, b+deltax, deltax).reshape(1,-1)
VJ = np.sin(np.pi * XJ0)

In [90]:
def cjkfun(VJ, J, j0, a, b, eps0):
    """
    输入：
        V0 : ndarray(1, n) -- 待插值函数
        J : int -- 小波插值层数
        j0 : int -- 初始层数
        a, b : int -- 函数的限制范围
        eps0 : float -- 阈值
    得到：
        cjk : list[ndarray(-1), ...] -- j层每层的所有\alpha不为0的配点位置索引
        cjknum : list -- 层配点索引数量（每层需计算的配点数量）
        zJc : list -- \alpha不为0的配点实际位置
        zJcnum : int -- 实际位置的数量
    """
    k0 = np.arange(0, 2**j0 + 1).reshape(1,-1)
    
    nn = k0 * (2**(J-j0))
    ff2 = VJ[0,nn[0]].reshape(1,-1)

    cjk0 = []
    cjk = [np.array([0])]
    cjknum = []
    zJc0 = nn[0]
    
    for j in range(j0, J):

        # j层的所有k索引的单变量，向量形式构建，一次性全部算出
        deltaj = (b-a)/2/(2**j)
        k = np.arange(0, 2**j).reshape((1,-1))

        # 这一层所有的x_{j+1, 2k+1}
        x = a + (2 * k + 1) * deltaj

        # \phi_{j_0, k_0}(x_{j+1,2k+1})
        theta = xphi0(j0, k0, a, b, x)
        # \sum\limits^{2^{j0}}_{k_0=0}u(x_{j_0k_0})\phi_{j_0, k_0}(x_{j+1,2k+1})
        s1 = np.dot(ff2 , theta.T)

        s2 = 0
        # \sum\limits^{j-1}_{j_1=j_0}
        for j1 in range(j0, j):
            # j1层的所有k1索引的单变量，向量形式构建，一次性全部算出
            k1 = np.arange(0, 2 ** j1).reshape((1,-1))
            # \psi_{j_1 k_1}(x_{j+1,2k+1}) == \phi_{j_1 + 1, 2*k_1 + 1}(x_{j+1,2k+1})
            theta = xphi0(j1 + 1,2 * k1 + 1,a,b,x)
            # \sum\limits_{k_1 \in Z^j_1}\alpha_{j_1 k_1}\psi_{j_1 k_1}(x_{j+1,2k+1})
            s2 = s2 + np.dot(cjk0[j1][0][k1], theta.T)  # cjk0[j1][0][k1] : 即\alpha_{j_1 k_1}

        # u(x_{j+1, 2k+1})
        nn = (2*k+1) * (2**(J-(j+1)))
        ff1 = VJ[0,nn[0]].reshape((1,-1))  

        # \alpha_{jk} = u(x_{j+1, 2k+1}) - [...+...]
        cjk0.append(ff1 - s1 - s2)

        # 统计大于阈值的\alpha_{jk}的值（减少无用计算）【自适应？？？】
        cjkvalue = abs(ff1 - s1 - s2)
        kk = np.flatnonzero(cjkvalue >= eps0)
        num = kk.shape[0]
        if num > 0:
            # 记录\alpha不为0的配点位置索引
            cjk.append(kk)
            # 记录\alpha不为0的配点实际位置
            zJc0 = np.append(zJc0, (2 * kk + 1) * (2 ** (J - j - 1)), 0)
        
        # 层配点索引数量（每层需计算的配点数量，去掉无用计算）
        cjknum.append(num)

        # 如果后面没有不为0的\alpha值了就没有计算下去的必要了
        if j >  j0 + 2 and num == 0:
            cjknum += [0]*(J-j-1)
            break
    # \alpha不为0的配点实际位置，从小到大排序方便索引
    zJc = np.sort(zJc0).tolist()
    # 实际位置的数量 其实== sum(cjknum) + 2 即两头+配点的数量总和
    zJcnum = len(zJc)

    # j层每层的所有\alpha不为0的配点位置索引
    # 层配点索引数量（每层需计算的配点数量）
    # \alpha不为0的配点实际位置
    # 实际位置的数量
    return cjk, cjknum, zJc, zJcnum
 

In [91]:
cjk, cjknum, zJc, zJcnum = cjkfun(VJ, J, j0, a, b, eps0)